<a href="https://colab.research.google.com/github/nimasajedi1/ReFED-Code-Challenge/blob/main/ReFED_Code_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

